In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import AutoConfig

checkpoint = "felflare/bert-restore-punctuation"
config = AutoConfig.from_pretrained(checkpoint)
print(config)

In [8]:
from transformers import BertTokenizer

checkpoint = "felflare/bert-restore-punctuation"
tokenizer = BertTokenizer.from_pretrained(checkpoint)

In [37]:
raw_inputs = ["in 2018 cornell researchers built a high-powered detector that in combination with an algorithm-driven process called ptychography set a world record by tripling the resolution of a state-of-the-art electron microscope as successful as it was that approach had a weakness it only worked with ultrathin samples that were a few atoms thick anything thicker would cause the electrons to scatter in ways that could not be disentangled now a team again led by david muller the samuel beckert professor of engineering has bested its own record by a factor of two with an electron microscope pixel array detector empad that incorporates even more sophisticated 3d reconstruction algorithms the resolution is so fine-tuned the only blurring that remains is the thermal jiggling of the atoms themselves"]
inputs = tokenizer(raw_inputs, return_tensors="pt")

In [27]:
from transformers import BertForTokenClassification

checkpoint = "felflare/bert-restore-punctuation"
model = BertForTokenClassification.from_pretrained(checkpoint)

In [28]:
outputs = model(**inputs)

In [29]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[[1.6809e-01, 5.6617e-01, 3.3986e-02,  ..., 6.5761e-03,
          7.3991e-03, 1.8238e-03],
         [9.9448e-01, 3.4018e-04, 3.7364e-05,  ..., 5.9698e-05,
          1.1287e-04, 2.2665e-04],
         [1.3555e-04, 1.5587e-01, 1.7322e-03,  ..., 1.1202e-03,
          5.9281e-04, 2.8866e-05],
         ...,
         [6.2292e-04, 9.9739e-01, 1.5162e-04,  ..., 2.0648e-05,
          1.4590e-05, 6.7091e-07],
         [2.8023e-04, 2.2620e-01, 6.4297e-01,  ..., 7.4284e-03,
          5.5669e-04, 2.0336e-05],
         [2.1335e-01, 1.8468e-01, 7.7130e-02,  ..., 3.5172e-02,
          3.1971e-02, 1.8259e-02]]], grad_fn=<SoftmaxBackward0>)


In [30]:
preds = torch.argmax(predictions,dim=-1).numpy()
print(preds)

[[1 0 4 0 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 7 7 4 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 2 0 1 1 1 4 1 1 1 1 2 0 1 1 1 1 1 1 1 1 1 1 1 1 2 0 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 2 2 2 7 1 1 1 1 1 0 7 1 0 0 0 0 1 7 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 0 1 1 1 1 1 4 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 2 0]]


In [76]:
    def punctuate_texts(full_pred: list):
        """
        Given a list of Predictions from the model, applies the predictions to text,
        thus punctuating it.
        """
        punct_resp = ""
        for i in full_pred:
            word, label = i            
            if label[-1] == "U":
                punct_wrd = word.capitalize()
            else:
                punct_wrd = word
            if label[0] != "O":
                punct_wrd += label[0]
            punct_resp += punct_wrd + " "   
        punct_resp = punct_resp.strip()
        # Append trailing period if doesnt exist.
        if punct_resp[-1].isalnum():
            punct_resp += "."
        return punct_resp

In [77]:
valid_labels = ['OU', 'OO', '.O', '!O', ',O', '.U', '!U', ',U', ':O', ';O', ':U', "'O", '-O', '?O', '?U']
for n in range(len(raw_inputs)):
  full_pred = []
  words = raw_inputs[n].split()
  fertility = [len(tokenizer.encode(x, add_special_tokens=False)) for x in words]
  j=1
  for i,word in enumerate(words):
    full_pred.append((word,valid_labels[preds[n][j]]))
    j+=fertility[i]
  print(punctuate_texts(full_pred))


In 2018, Cornell researchers built a high-powered detector that, in combination with an algorithm-driven process called Ptychography, set a world record by tripling the resolution of a state-of-the-art electron microscope. As successful as it was, that approach had a weakness. It only worked with ultrathin samples that were a few atoms thick. Anything thicker would cause the electrons to scatter in ways that could not be disentangled. Now, a team again led by David Muller, the Samuel Beckert Professor of Engineering, has bested its own record by a factor of two with an electron microscope pixel array detector empad that incorporates even more sophisticated 3d reconstruction algorithms. The resolution is so fine-tuned the only blurring that remains is the thermal jiggling of the atoms themselves.
